This notebook is about Bokeh and 2-way communication (synchronization) using a Bokeh server. While One-way communication with the widgets only updating the plots can be established with a Bokeh server, that is not what this notebook is about. 

Synchronization between a widgetted Bokeh plot and the variables within the underlying Python environment requires the use of a Bokeh server and function callbacks. More info at https://docs.bokeh.org/en/latest/docs/user_guide/server.html#python-callbacks-with-jupyter-interactors 

See example at https://github.com/bokeh/bokeh/blob/2.4.2/examples/howto/server_embed/notebook_embed.ipynb 

In [11]:
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import CDSView, ColumnDataSource, IndexFilter, RadioGroup, Slider, BoxEditTool
from bokeh.io import show, output_notebook
from bokeh import palettes

In [12]:
output_notebook()

Loading BokehJS ...

In [4]:
import random
from functools import partial
import numpy as np
import pandas as pd

In [5]:
# Get some data into a dataframe
x1 = list(range(0, 10))
y1 = random.sample(range(0, 10), 10)
# Prepare an output dataframe that will contain the labels (initially, there is none)
df = pd.DataFrame({'x':x1, 'y':y1, 'label': 'no label'})
df.set_index('x', inplace=True)

In [6]:
# Create your labels
labels = ['class 1', 'class 2', 'class 3']
# Create associated visual aid: e.g. 1 color per class. 
colors = ['red', 'black', 'blue']
# Initialize a basic container of the labelled datasets (list of 3 lists)
clusters = [[], [], []]

In [18]:
# Define the app. The "doc" argument is the container of your app. We add to it the graphs and GUI elements
def bkapp1(doc):
    # Import your data in the Bokeh source data object. Similar to Pandas dataframe, which can also be imported.
    source = ColumnDataSource(dict(x1=x1, y1=y1))
    # Define the tools that will appear in the figure toolbar. 
    # The selection tool is necessary, e.g. `box_select`.
    tools = ['box_select', 'hover', 'reset']
    # Create the figure axis
    plot = figure(x_axis_label='x',
                  y_axis_label='Temperature (Celsius)',
                  title="Test saving selection for labelling",
                  tools=tools)
    
    source1 = ColumnDataSource({'x':[], 'y':[], 'width':[], 'height':[]})
    source2 = ColumnDataSource({'x':[], 'y':[], 'width':[], 'height':[]})
    r1 = plot.rect('x', 'y', 'width', 'height', source=source1)
    r2 = plot.rect('x', 'y', 'width', 'height', source=source2)
    tool = BoxEditTool(renderers=[r1, r2], num_objects=2)
    plot.add_tools(tool)
    
    # Add the data renderer (here a scatter plot)
    scatter = plot.scatter(x='x1', y='y1', source=source, 
                           selection_color = 'firebrick',
                           nonselection_fill_alpha=0.4)
    # For manually labelling, use a group of radio buttons for mutually exclusive classes (one buttons checked at a time)
    radio_group = RadioGroup(labels=["Cluster 1", "Cluster 2", "Cluster 3"], active=0)

    # Define a function that will run when the button is clicked (aka handler)
    def my_radio_handler(new):
        print(f'Radio button {new} selected')

    # Assign it to the "on_click" action of the radio group instance
    radio_group.on_click(my_radio_handler)
    # Below we get to the most important interaction function: the "callback" function. 
    # Callback functions define what happens when you interact with your data through a widget. 
    # they are defined just as `callback(attr, old, new)`, where `attr` refers to the changed 
    # attribute’s name, and `old` and `new` refer to the previous and updated values of the attribute.
    # The execution of the callback function will be triggered by the `on_change()` method 
    # of the `scatter.data_source` (see down below)
    
    def callback1(attr, old, new):
        # Let's use some shared variables. More elegant ways may exist. 
        global clusters, colors, labels
        # Get which radio button (the widget instance) is clicked, or `active`: this is an integer number used as index
        a = radio_group.active
        # `new` is the new values taken by the attribute of the callback. 
        # Here, that is the list of indices of our newly selected data
        # We append that list into our global output variable
        clusters[a] = clusters[a] + new
        # Assign the label in the dataframe. Direct index selections avoids the problem of redundant selection.
        df.loc[clusters[a], 'label'] = labels[a]

        # Plot a so-called "view" to overlay the selected data as we go, using "IndexFilter" to avoid hard copies
        view = CDSView(source=source, filters=[IndexFilter(clusters[a])])
        plot.scatter(x='x1', y='y1', source=source, view=view,
                     legend_label=f'{labels[a]}', # dynamic legend, to show a new legend when we add a new label
                     size=20, line_width=2,
                     fill_color=None, line_color=colors[a],
                     nonselection_fill_alpha=0.4,
                     nonselection_line_alpha=1.0)

        # Display how many we selected in each class, just as a visual check
        nclasses = [len(df[df['label'] == s]) for s in labels]
        plot.title.text = f'Class 1: {nclasses[0]} -- Class 2: {nclasses[1]} -- Class 3: {nclasses[2]}'
        

    # Our callback function is complete. Let's send it to the `on_change()` method
    # More info at: https://docs.bokeh.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnarDataSource.on_change
    # The interaction between the callback and the widget depends on the state of the widget (our radio buttons)
    # so we must query those states => The button objects must be known by this function. 
    # This kind of interaction is not explicitly documented in the Bokeh intro tutorials: 
    # In fact, nothing prevents you from passing the callback function to the "trigger" functions (e.g. `on_change()` like below) 
    # with more objects. This is possible by compiling a `partial` callback function, using `functools.partial()`. 
    # Here, we will pass our radio buttons.
    scatter.data_source.selected.on_change('indices', callback1)
    
    # Add our GUI and graph in a column layout
    doc.add_root(column(radio_group, plot))

In [19]:
# Start and show the app
show(bkapp1)

In [40]:
df

,y,label
x,,
0,9,no label
1,3,no label
2,7,no label
3,0,no label
4,6,no label
5,4,no label
6,2,no label
7,1,no label
8,5,no label


In [1]:
# Define the app. The "doc" argument is the container of your app. We add to it the graphs and GUI elements
def bkapp(doc):
    N1 = 600
    N2 = 659
    x = np.linspace(0, 10, N1)
    y = np.linspace(0, 10, N2)
    xx, yy = np.meshgrid(x, y)
    # Random periodic data with noise
    noise_data = [np.sin(xx)*np.cos(yy) + np.random.rand(N2,N1) for _ in range(10)]
    
    tools = ['box_select', 'hover', 'reset']
    myslider = Slider(title="amplitude", value=0, start=0, end=10, step=1)
    # Create the figure axis
    p = figure(x_axis_label='X [px]',
                  y_axis_label='Y [px]',
                  title="Test displaying image series",
                  tools=tools)
    
    p.x_range.range_padding = p.y_range.range_padding = 0
    # Display the image
    p.image(image=[noise_data[myslider.value]], x=0, y=0, dw=N1, dh=N2, palette=palettes.Greys256)
   
        
    def callback(attr, old, new):
        p.image(image=[noise_data[myslider.value]], x=0, y=0, dw=N1, dh=N2, palette=palettes.Greys256)
        print(myslider.value)

    # scatter.data_source.selected.on_change('indices', callback1)
    myslider.on_change('value', callback)
    
    # Add our GUI and graph in a column layout
    doc.add_root(column(myslider, p))

In [ ]:
show(bkapp)

In [20]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import curdoc
from bokeh.layouts import column, row, layout
from bokeh.models import ColumnDataSource
from bokeh.models.tools import BoxEditTool


COLORS = ["red","blue"]

output_file("box_edit.html")

p = figure(plot_width=400, plot_height=400)

source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y=[2, 5, 8, 2, 7],
))

p.circle('x', 'y', size=20, source=source)

sectordata1 = ColumnDataSource(
    data={'x': [], 'y': [], 'width': [], 'height': []})
sectordata2 = ColumnDataSource(
    data={'x': [], 'y': [], 'width': [], 'height': []})

r1 = p.rect('x', 'y', 'width', 'height', color=COLORS[0], source=sectordata1)
r2 = p.rect('x', 'y', 'width', 'height', color=COLORS[1], source=sectordata2)
tool = BoxEditTool(renderers=[r1, r2],num_objects=2)
p.add_tools(tool)
show(p)